# 🌿 Plant Disease Detection Using CNN
This notebook trains a Convolutional Neural Network (CNN) to detect plant diseases using leaf images.
We use the PlantVillage dataset and TensorFlow/Keras for model training.

In [ ]:
# 📦 Install required libraries
!pip install tensorflow keras matplotlib opencv-python

## 📁 Dataset Preparation
- Download the PlantVillage dataset from Kaggle: https://www.kaggle.com/datasets/emmarex/plantdisease
- Unzip it and make sure the folder is named `plant_dataset` and structured like this:
```
plant_dataset/
    ├── Apple___Black_rot/
    ├── Apple___healthy/
    ├── Tomato___Late_blight/
    └── Tomato___healthy/
```

In [ ]:
# 📚 Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

In [ ]:
# 🔄 Preprocess dataset
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    'plant_dataset',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_data = train_datagen.flow_from_directory(
    'plant_dataset',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

In [ ]:
# 🧠 Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 🚀 Train model
history = model.fit(train_data, validation_data=val_data, epochs=10)

In [ ]:
# 💾 Save model
model.save("plant_disease_model.h5")

In [ ]:
# 📈 Plot accuracy & loss
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')
plt.show()

In [ ]:
# 🔍 Predict on new image
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img("test_leaf.jpg", target_size=(128, 128))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)
class_labels = list(train_data.class_indices.keys())
print("Predicted disease:", class_labels[predicted_class])